In [ ]:
import gzip

import pandas as pd

pd.set_option("display.max_columns", None)

Read LOH positions

In [ ]:
df = pd.read_csv("../intersect_mmml_onek1k_with_header.vcf.gz", sep="\t", compression="gzip", low_memory=False)
len(df)

In [ ]:
df = df.drop(columns=["Unnamed: 29", "position_dummy", "#CHR", "POS", "POS_DUMMY"])

In [ ]:
df["position_code"] = df["#CHROM"].astype(str) + "-" + df["pos"].astype(str)

In [ ]:
df.head()

Analyse LOH positions

In [ ]:
no_distinct_positions = df["position_code"].nunique()
no_distinct_positions

In [ ]:
df["#CHROM"].nunique()

In [ ]:
no_distinct_patients = df["PID"].nunique()
no_distinct_patients

In [ ]:
unique_combinations = len(df.drop_duplicates(['PID','position_code']).index)
unique_combinations

Merge position count

In [ ]:
ser = pd.read_json("../intersect_position_count.json", typ="series", dtype={str})
df_position_count = ser.to_frame("position_count")
df_position_count = df_position_count.reset_index(names="pos")
df_position_count.shape

In [ ]:
df_position_count.head()

In [ ]:
df = df.merge(df_position_count, how="inner", left_on="position_code", right_on="pos")
len(df)

Set threshold

In [ ]:
threshold = 40
df_thresholded = df[df["position_count"] >= threshold]
len(df_thresholded)

Analyze thresholded dataframe

In [ ]:
df_thresholded["PID"].nunique()

In [ ]:
df_thresholded["#CHROM"].nunique()

In [ ]:
df_thresholded["#CHROM"].unique()

In [ ]:
df_thresholded["position_code"].nunique()

Write to csv

In [ ]:
df_thresholded.to_csv(f"../LOH_pos_thresholded/LOH_positions_threshold_{threshold}.tsv.gz", sep="\t", header=True, index=False, compression="gzip")